## Interactivity in plotly

The goal of this notebook is to understand basic interactivity in plotly, ie the capability to update a plot based on mouse events on the same plot. 

This feature might not be available in all plotly renderers, but it should work in the default one (plotly_mimetype). In order to activate the plotly notebook renderer, you can run the following code:

```python
import plotly.io as pio
pio.renderers.default = "plotly_mimetype"
```


## Setup environment

For this example, we'll use the same data as in examples 1 and 3. 

In [2]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio
pio.renderers.default = "plotly_mimetype"

In [3]:
FILE = "data/ElectricVehicle.xlsx"

In [4]:
df = pd.read_excel(FILE)
df.columns = ['distance_electric', 'fuel', 'electric', 'total_distance', 'speed', 'energy_recovered']
df['charge'] = df.distance_electric / df.total_distance
df.head()

,distance_electric,fuel,electric,total_distance,speed,energy_recovered,charge
0,27.7,0.3,12.0,29.4,43.3,NaN,0.942177
1,24.1,0.5,14.9,24.9,14.1,NaN,0.967871
2,1.9,4.7,3.2,21.9,22.4,NaN,0.086758
3,0.1,5.9,5.6,21.8,16.8,2.1,0.004587
4,2.1,4.7,3.8,32.5,19.1,3.1,0.064615


## Use case: clustering

Let's perform a clustering of the data using sklearn. 

In [5]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(df[['charge', 'electric']])

clusters = kmeans.predict(df[['charge', 'electric']])
df['cluster'] = clusters

## Plotting the clusters

We can produce a scatterplot in plotly where the color indicates the cluster number: 

In [6]:
figure = go.Figure(
    go.Scatter(
        x=df.charge,
        y=df.electric,
        mode='markers',
        marker=dict(
            size=8,
            color=df.cluster,
            line=dict(
                width=2,
                color='blue'
            )
        ),
    )   
)
figure.update_layout(
    title='Electric Vehicle',
    xaxis_title='Initial charge',
    yaxis_title='Electric consumption (kWh/100km)'
)




## Callbacks

Say we now want to highlight the clusters in an interactive way.

Each time the mouse moves over a point, its cluster should be highlighted (and the other clusters "turned off"). 

This can be useful for a dashboard for data exploration or in a business understanding task. 

### Method

- To add interactivity to a plotly plot, we can use a special type of figure object, `go.FigureWidget`. 
- It includes methods to specify what happens for certain mouse events, such as `on_click` or `on_hover`. 
- The argument for these function is the name of another function (sometimes known as callback). 
- The signature of the callback function is `function_name(trace, points, selector)`. `trace`, `points` and `selector` can be used to track different values related to the mouse event. For example, `points` is a structure that contains information about the points of the plot involved in the event. In the following example, `points.inds` is a list of the indices all the points involved in the event (in that case there should be only 1 point). 
- The context manager `batch_update` will apply updates to the plot as needed but only render the plot when finished, avoiding to render the plot for each change. 

In [8]:
import numpy as np
widget = go.FigureWidget(figure)

def update_point(trace, points, selector):

    for i in points.point_inds:
        cluster_id = df.cluster[i]
        # Get all indices of the same cluster as integer
        cluster_indices = np.where(df.cluster == cluster_id)[0]
        opacity = np.array([0.1] * len(df))
        opacity[cluster_indices] = 1

        with widget.batch_update():
            widget.data[0].marker.opacity = opacity
            

widget.data[0].on_hover(update_point)

widget.show()

FigureWidget({
    'data': [{'marker': {'color': {'bdata': ('AAAAAAAAAAABAAAAAQAAAAEAAAABAA' ... 'ABAAAAAAAAAAIAAAACAAAAAAAAAA=='),
                                   'dtype': 'i4'},
                         'line': {'color': 'blue', 'width': 2},
                         'size': 8},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '86bff3cb-cbff-4db2-870c-651985269384',
              'x': {'bdata': ('iivdG1Am7j+FgFufzfjuP1xjcY3FNb' ... 'X6WDrgP/myQZMvG+Q/QW23mr7R6T8='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAAAAKEDNzMzMzMwtQJqZmZmZmQ' ... 'MzMzMiQM3MzMzMzCFAmpmZmZmZKEA='),
                    'dtype': 'f8'}}],
    'layout': {'template': '...',
               'title': {'text': 'Electric Vehicle'},
               'xaxis': {'title': {'text': 'Initial charge'}},
               'yaxis': {'title': {'text': 'Electric consumption (kWh/100km)'}}}
})

# Common use case: dashboard

A common use case of callbacks in plotly is having a click in one plot update a property of another plot. 

In [9]:
DATA = "data/hotel_bookings.csv"
df2 = pd.read_csv(DATA)


In [11]:
# Produce two subplots in the same row

from plotly.subplots import make_subplots

CATEGORIES = ['Transient', 'Contract', 'Transient-Party', 'Group']

df2['customer_category'] = pd.Categorical(df2['customer_type'], categories=CATEGORIES, ordered=True)

fig = go.FigureWidget(
    make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5]
    )
)

fig.add_histogram(x=df2['customer_category'], histnorm="percent", row=1, col=1)
fig.add_box(x=df2['is_canceled'], y=df2['lead_time'], row=1, col=2)

fig['layout']['xaxis1']['title']='Type of customer'
fig['layout']['xaxis2']['title']='Has cancelled'
fig['layout']['yaxis1']['title']='Percentage'
fig['layout']['yaxis2']['title']='Lead time'

fig.update_layout(
    showlegend=False,
)

fig

FigureWidget({
    'data': [{'histnorm': 'percent',
              'type': 'histogram',
              'uid': 'ad9a1196-e995-4d98-a445-66219c8c7004',
              'x': array(['Transient', 'Transient', 'Transient', ..., 'Transient', 'Transient',
                          'Transient'], dtype=object),
              'xaxis': 'x',
              'yaxis': 'y'},
             {'type': 'box',
              'uid': 'cb12a9ee-acf1-456a-ab3f-833ff494c86e',
              'x': {'bdata': ('AAAAAAAAAAABAQEAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
                    'dtype': 'i1'},
              'xaxis': 'x2',
              'y': {'bdata': ('VgHhAgcADQAOAA4AAAAJAFUASwAXAC' ... 'AsALwAhwCkABUAFwBmACIAbQDNAA=='),
                    'dtype': 'i2'},
              'yaxis': 'y2'}],
    'layout': {'showlegend': False,
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.45], 'title': {'text': 'Type of customer'}},
               'xaxis2': {'anchor': 'y2', 'do

In [12]:
def update_right_plot(trace, points, selector):
    if not points.point_inds:
        return

    with fig.batch_update():
        selected_index = points.point_inds[0]
        selected_customer_type = df2.customer_type[selected_index]
        selected_data = df2[df2.customer_type == selected_customer_type]

        # Get the index of the selected customer type
        selected_index = CATEGORIES.index(selected_customer_type)
        opacity = ["lightgrey"] * 4
        opacity[selected_index] = "blue"

        fig.data[0].marker.color = opacity
        fig.data[1].x = selected_data['is_canceled']
        fig.data[1].y = selected_data['lead_time']

fig.data[0].on_click(update_right_plot)
